<a href="https://colab.research.google.com/github/thazin31086/WebTrace/blob/master/UniversalSentenceEncoderRoslyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import keras
import numpy as np
import re

from sklearn import preprocessing


url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsRoslyn.csv')

y = list(data['Label']) 
x = list(data['Title']) 


#Clean Title 
for title in x:
    re.sub("[^a-zA-Z]+", "", title)

le = preprocessing.LabelEncoder()
le.fit(y)


categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[0:900])
y_train = np.asarray(y_enc[0:900])

x_test = np.asarray(x_enc[0:200])
y_test = np.asarray(y_enc[0:200])

len(x_train)

#average_length = np.mean([len(x) for x in x_train])
#median_length = sorted([len(x) for x in x_train])[len(x_train) // 2]

#print("Average sequence length",average_length)
#print("Median sequence length",median_length)

max_sequence_length = 180


from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))


input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [25]:
max_sequence_length = 512

from keras.preprocessing import sequence

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=30, batch_size=32)
    model.save_weights('./model.h5')

Epoch 1/30
900/900 [==============================] - 11s 12ms/step - loss: 5.1267 - acc: 0.1778
Epoch 2/30
900/900 [==============================] - 6s 7ms/step - loss: 3.8747 - acc: 0.2322
Epoch 3/30
900/900 [==============================] - 7s 7ms/step - loss: 3.5015 - acc: 0.3467
Epoch 4/30
900/900 [==============================] - 7s 8ms/step - loss: 3.3069 - acc: 0.3500
Epoch 5/30
900/900 [==============================] - 7s 7ms/step - loss: 3.1516 - acc: 0.3678
Epoch 6/30
900/900 [==============================] - 7s 8ms/step - loss: 3.0168 - acc: 0.3711
Epoch 7/30
900/900 [==============================] - 7s 7ms/step - loss: 2.8825 - acc: 0.3722
Epoch 8/30
900/900 [==============================] - 7s 7ms/step - loss: 2.7583 - acc: 0.3844
Epoch 9/30
900/900 [==============================] - 7s 8ms/step - loss: 2.6330 - acc: 0.4011
Epoch 10/30
900/900 [==============================] - 7s 7ms/step - loss: 2.5071 - acc: 0.4078
Epoch 11/30
900/900 [==========================

In [32]:
len(x)


1201

In [35]:
x_test = np.asarray(x_enc[0:200])
y_test = np.asarray(y_enc[0:200])

len(x_test)

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model.h5')  
    predicts = model.predict(x_test, batch_size=32)

    
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))

                                                                                                                             precision    recall  f1-score   support

       1.00      1.00      1.00         3
                                                 4 - In Review|Area-IDE|Bug|New Language Feature - Nullable Reference Types       0.00      0.00      0.00         0
                                                                                     4 - In Review|Area-IDE|Feature Request       1.00      1.00      1.00         1
                                                                                Approved to merge|Area-IDE|Area-Interactive       1.00      1.00      1.00         1
                                                                                              Area-Analyzers|Area-Compilers       0.80      0.80      0.80         5
                                                                          Area-Analyzers|Area-Compilers|Concept-Design Debt       1.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# New Section